In [2]:
import pandas as pd
"""
Right now we are downloading data from 
https://fragilestatesindex.org/
Then processing it for move up to red
We need to automate this process for use in our new green to red pipeline(WIP)
On a monthly basis we need to do a data pull and get any new data
If we put it in a dash app we can use POSIT CONNECT to keep the script persisting
"""
df = pd.read_excel("Rankings.xlsx")

cols = []
for col in df.columns:
    if "Unnamed" in col:
        cols.append(df[col].iloc[0])
    else:
        cols.append(col)
        
df.columns = cols
df = df[1:]

df = df[["Country", "Total"]]
df.columns = ["Country", "Total"]
df["Year"] = 2024


In [3]:
df

,Country,Total,Year
1,Somalia,111.3,2024
2,Sudan,109.3,2024
3,South Sudan,109.0,2024
4,Syria,108.1,2024
5,Congo Democratic Republic,106.7,2024
...,...,...,...
175,Denmark,15.9,2024
176,New Zealand,15.9,2024
177,Iceland,15.2,2024
178,Finland,14.3,2024


In [4]:
df.to_excel("FSI_output.xlsx")

In [5]:
import dash
from dash import Dash, dcc, html, Input, Output
import matplotlib.pyplot as plt
import plotly.express as px
import requests

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# Path to your ChromeDriver
# Set up the Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Optional: run in headless mode
driver = webdriver.Chrome(options=options)

url = "https://fragilestatesindex.org/excel/"
driver.get(url)

# Wait for the page to load (adjust time if necessary)
driver.implicitly_wait(10)

# Extract links
links = set()
for link in driver.find_elements(By.TAG_NAME, "a"):
    href = link.get_attribute("href")
    if href and (".xlsx" in href or ".xls" in href):
        links.add(href)

print("Excel Links Found:", links)

driver.quit()

Excel Links Found: {'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2017.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/2021/05/fsi-2021.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2008.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2006.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/2023/06/FSI-2023-DOWNLOAD.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/2018/04/fsi-2018.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2014.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/2019/04/fsi-2019.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2016.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2015.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2012.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2010.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2009.xlsx', 'https://fragilestatesi

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By

# Set up the Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Optional: run in headless mode
driver = webdriver.Chrome(options=options)

# Access the website
driver.get("https://fragilestatesindex.org/excel")

# Extract data
links = driver.find_elements(By.XPATH, "//a[contains(@href, 'https://fragilestatesindex.org/wp-content/uploads/')]")
links_set = set()

for link in links:
    links_set.add(link.get_attribute("href"))

print(links_set)
driver.quit()

{'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2017.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/2021/05/fsi-2021.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2008.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2006.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/2023/06/FSI-2023-DOWNLOAD.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/2018/04/fsi-2018.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2014.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/2019/04/fsi-2019.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2016.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2015.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2012.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2010.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2009.xlsx', 'https://fragilestatesindex.org/wp-content

In [8]:
from io import BytesIO

combined_df = pd.DataFrame()

# Define headers to mimic a browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

columns_to_keep = ["Country", "Year", "Total"]

# Loop through each link, download the file, and read it
for link in links_set:
    try:
        # Send GET request with headers
        response = requests.get(link, headers=headers)
        response.raise_for_status()  # Raise an error for bad status codes
        
        # Read the Excel file into a DataFrame
        df = pd.read_excel(BytesIO(response.content), usecols=columns_to_keep)
        
        # Append the data to the combined DataFrame
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    except Exception as e:
        print(f"Failed to process {link}: {e}")

combined_df['Year'] = combined_df['Year'].astype(str).str[:4].astype(int)
output_file = "FSI_Combined.xlsx"
combined_df.to_excel(output_file, index=False)

# Display the combined DataFrame
print(combined_df)

                       Country  Year       Total
0                  South Sudan  2017  113.900000
1                      Somalia  2017  113.400000
2     Central African Republic  2017  112.600000
3                        Yemen  2017  111.100000
4                        Sudan  2017  110.600000
...                        ...   ...         ...
3165                   Iceland  2020   17.800000
3166                   Denmark  2020   17.213587
3167               Switzerland  2020   17.094086
3168                    Norway  2020   16.191210
3169                   Finland  2020   14.626666

[3170 rows x 3 columns]


In [9]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Line Chart by Country"),
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in combined_df['Country'].unique()],
        value='United States',  # Default value
        placeholder="Select a country"
    ),
    dcc.Graph(id='line-chart')
])

# Callback to update the graph based on selected country
@app.callback(
    Output('line-chart', 'figure'),
    Input('country-dropdown', 'value')
)
def update_chart(selected_country):
    # Filter data by selected country
    filtered_df = combined_df[combined_df['Country'] == selected_country]
    # Sort data by year
    filtered_df = filtered_df.sort_values(by='Year')
    # Create the line chart
    fig = px.line(filtered_df, x='Year', y='Total', title=f'Total by Year for {selected_country}')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)